## 뉴스 Crawling ㄱㄱ

In [61]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time
import re

# --------------------------------------
# 웹페이지 열고 이동하기 
# --------------------------------------
driver = Chrome()

# 브라우저 크기 최대화
driver.maximize_window()

# --------------------------------------
# 종목 코드 이용하여 주식 뉴스 페이지 들어가기
# --------------------------------------
stockCode = '005930' 	# 삼성전자 종목 코드
#startPage = 1			# 뉴스 수집 시작 페이지
lastPage = 500 			# 뉴스 수집 마지막 페이지


url = f'https://finance.daum.net/quotes/A{stockCode}#news/stock'
driver.get(url)

In [ ]:
news_data = []

try:
    current_page = 1
    max_page = lastPage

    while current_page <= max_page:
        group_start = ((current_page - 1) // 10) * 10 + 1
        group_end = min(group_start + 9, max_page)
        
        print(f"페이지 그룹 {group_start}-{group_end} 크롤링 시작...")

        for page_in_group in range(group_start, group_end + 1):
            print(f"크롤링 중: {page_in_group} 페이지")
            
            # 그룹의 첫 페이지가 아닌 경우에만 클릭
            if page_in_group != group_start:
                try:
                    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.paging')))
                    time.sleep(0.3)
                    pagination = driver.find_element(By.CSS_SELECTOR, 'div.paging')
                    page_links = pagination.find_elements(By.TAG_NAME, 'a')
                    
                    page_clicked = False
                    for link in page_links:
                        try:
                            if int(link.text) == page_in_group:
                                driver.execute_script("arguments[0].click();", link)
                                time.sleep(0.2)
                                page_clicked = True
                                break
                        except ValueError:
                            continue
                    if not page_clicked:
                        print(f"페이지 {page_in_group} 링크를 찾을 수 없습니다.")
                        continue
                except Exception as e:
                    print(f"페이지 {page_in_group} 이동 실패: {e}")
                    continue

            # 기존 크롤링 코드 (그대로 유지)
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'li.imgB')))
                time.sleep(0.2)

                news_items = driver.find_elements(By.CSS_SELECTOR, 'li.imgB')

                for item in news_items:
                    try:
                        link_tag = item.find_element(By.CSS_SELECTOR, 'a.tit')
                        newsTitle = link_tag.text.strip()
                        link = link_tag.get_attribute('href')
                        txt_tag = item.find_element(By.CSS_SELECTOR, 'a.txt')
                        newsText = txt_tag.text.strip()
                        date_tag = item.find_element(By.CSS_SELECTOR, 'p.date')
                        date_text = date_tag.text.strip()
                        if '·' in date_text:
                            newsFrom, newsDate = map(str.strip, date_text.split('·', 1))
                        else:
                            newsFrom, newsDate = '', date_text
                        news_data.append({'date': newsDate, 'title': newsTitle, 'text': newsText, 'from': newsFrom, 'link': link})
                    except Exception:
                        continue
            except Exception as e:
                print(f"페이지 {page_in_group} 크롤링 실패: {e}")
                continue

        # 다음 페이지 그룹으로 이동 (btnNext 클릭)
        current_page = group_end + 1
        if current_page <= max_page:
            try:
                WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.paging')))
                pagination = driver.find_element(By.CSS_SELECTOR, 'div.paging')
                next_btns = pagination.find_elements(By.CSS_SELECTOR, 'a.btnNext')
                if next_btns and next_btns[0].is_enabled():
                    print("다음 페이지 그룹으로 이동...")
                    driver.execute_script("arguments[0].click();", next_btns[0])
                    time.sleep(0.5)
                else:
                    print("더 이상 다음 페이지 그룹이 없습니다.")
                    break
            except Exception as e:
                print(f"다음 페이지 그룹 이동 실패: {e}")
                break

    news_df = pd.DataFrame(news_data)
    print(f"\n✅ 크롤링 완료! 총 {len(news_df)}개의 뉴스를 수집했습니다.")
    print(news_df)

except Exception as e:
    print(f"❌ 오류 발생: {e}")


페이지 그룹 1-10 크롤링 시작...
크롤링 중: 1 페이지
크롤링 중: 2 페이지
크롤링 중: 3 페이지
크롤링 중: 4 페이지
크롤링 중: 5 페이지
크롤링 중: 6 페이지
크롤링 중: 7 페이지
크롤링 중: 8 페이지
크롤링 중: 9 페이지
크롤링 중: 10 페이지
다음 페이지 그룹으로 이동...
페이지 그룹 11-20 크롤링 시작...
크롤링 중: 11 페이지
크롤링 중: 12 페이지
크롤링 중: 13 페이지
크롤링 중: 14 페이지
크롤링 중: 15 페이지
크롤링 중: 16 페이지
크롤링 중: 17 페이지
크롤링 중: 18 페이지
크롤링 중: 19 페이지
크롤링 중: 20 페이지
다음 페이지 그룹으로 이동...
페이지 그룹 21-30 크롤링 시작...
크롤링 중: 21 페이지
크롤링 중: 22 페이지
크롤링 중: 23 페이지
크롤링 중: 24 페이지
크롤링 중: 25 페이지
크롤링 중: 26 페이지
크롤링 중: 27 페이지
크롤링 중: 28 페이지
크롤링 중: 29 페이지
크롤링 중: 30 페이지
다음 페이지 그룹으로 이동...
페이지 그룹 31-40 크롤링 시작...
크롤링 중: 31 페이지
크롤링 중: 32 페이지
크롤링 중: 33 페이지
크롤링 중: 34 페이지
크롤링 중: 35 페이지
크롤링 중: 36 페이지
크롤링 중: 37 페이지
크롤링 중: 38 페이지
크롤링 중: 39 페이지
크롤링 중: 40 페이지
다음 페이지 그룹으로 이동...
페이지 그룹 41-50 크롤링 시작...
크롤링 중: 41 페이지
크롤링 중: 42 페이지
크롤링 중: 43 페이지
크롤링 중: 44 페이지
크롤링 중: 45 페이지
크롤링 중: 46 페이지
크롤링 중: 47 페이지
크롤링 중: 48 페이지
크롤링 중: 49 페이지
크롤링 중: 50 페이지
다음 페이지 그룹으로 이동...
페이지 그룹 51-60 크롤링 시작...
크롤링 중: 51 페이지
크롤링 중: 52 페이지
크롤링 중: 53 페이지
크롤링 중: 54 페이지
크롤링 중: 55 페이지
크롤링 중: 56 페이

In [63]:
# CSV로 저장 (선택사항)
uniqueTime = datetime.now().strftime('%Y%m%d_%H%M%S')
file_path_csv = f'data/{uniqueTime}_daumNews.csv'
#file_path_excel = f'data/{uniqueTime}_daumNews.xlsx'
news_df.to_csv(file_path_csv, index = False, encoding = "utf-8-sig")	
print(f"결과가 {file_path_csv} 파일로 저장되었습니다.")


결과가 data/20250602_192034_daumNews.csv 파일로 저장되었습니다.
